# "We'll Cross the Streams": Combining Asynchronous Data Streams

![Ghostbusters reference](https://media.giphy.com/media/3o72EWUgbRNfLegO1W/giphy.gif)

![Ghostbusters reference](https://thumbs.gfycat.com/ThirstyEnchantedCaudata-size_restricted.gif)

## Configuration

This code would normally go in a script automatically run at startup. The user would not have to worry about this.

In [ ]:
%run ../beamline_configuration.py

## Monitor current asynchronously while counting the detector.

Monitoring can be done on a one-off basis, but it's typically set up in a semi-permanent way: you want to "set it and forget it." The ``sd`` object keeps track of things to monitor concurrently with other measurements. We'll add the beam current signal ``I`` to that list.

In [ ]:
sd

In [ ]:
sd.monitors.append(I)

In [ ]:
RE(scan([slit], motor_slit, -15, 15, 150))

In [ ]:
header = db[-1]
header.table().head()  # shows the 'primary' stream by default

In [ ]:
header.table(stream_name='primary').head()  # equivalent to the above

What other streams are there?

In [ ]:
header.stream_names

In [ ]:
header.table('I_monitor').head()

### Plot data stream together

We can plot them each against _time_ but we cannot plot them directly against each other because they have different time bases. To compare them, we have to interpolate one at the time points of the other.

In [ ]:
plt.figure()
plt.plot('time', 'slit_det', data=header.table(), marker='o', label='slit_det')
plt.plot('time', 'I', data=header.table(stream_name='I_monitor'), marker='x', label='I')
plt.legend()

### Interpolate and normalize

Concatenate the tables side by side. The result is a sort of block matrix of missing data (NaN).

In [ ]:
import pandas as pd
data = pd.concat([header.table('primary'), header.table('I_monitor')], axis=0)
data

In [ ]:
# Make 'time' the index and sort on it.
data.set_index('time').sort_index().head(20)

In [ ]:
# Consider I alone
data.set_index('time').sort_index()['I']

In [ ]:
interp_data = data.set_index('time').sort_index().ffill()

In [ ]:
interp_I = data.set_index('time').sort_index()['I'].interpolate('linear')
interp_I.head(20)

In [ ]:
interp_data['normalized'] = interp_data['slit_det'] / interp_data['I'] * interp_data['I'].mean()

In [ ]:
plt.figure()
plt.plot('motor_slit', 'slit_det', data=interp_data, label='raw')
plt.plot('motor_slit', 'normalized', data=interp_data, label='interpolated and normalized')
plt.legend()

## Exercises

1. Execute the ``monitor_count`` with a different ``delay`` parameter to verify that the readings from ``sig`` come at 5 Hz.